In [ ]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
import datetime
import os
import numpy as np
import tensorflow as tf

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, confusion_matrix

In [ ]:
main_path = f"/content/gdrive/My Drive"
dataset = "ISIC 2018"
sub_path = f"{main_path}/Data/{dataset}/256/"

pred_file = "preds_EffnetB7_R2_deep5_2021-02-25 06:40:39.npz"

In [ ]:
Y_test = np.load(f"{sub_path}test_labels.npz")["arr_0"]

Y_test = Y_test / 255

In [ ]:
print(np.max(Y_test))
print(np.min(Y_test))

1.0
0.0


In [ ]:
Y_test.shape

(519, 256, 256)

In [ ]:
Y_test = np.reshape(Y_test, (Y_test.shape[0], Y_test.shape[1], Y_test.shape[2], 1))

In [ ]:
Y_test.shape

(519, 256, 256, 1)

In [ ]:
Y_pred = np.load(f"{main_path}/Data/{dataset}/256/predictions/{pred_file}")["arr_0"]

In [ ]:
Y_pred.shape

(519, 256, 256, 1)

In [ ]:
# Y_pred = tf.image.crop_to_bounding_box(Y_pred, offset, offset, target, target).numpy()

In [ ]:
Y_pred.shape

(519, 256, 256, 1)

In [ ]:
def dice_coef(y_true, y_pred):
    smooth = 0
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

In [ ]:
dice = dice_coef(Y_test, Y_pred)

In [ ]:
print(f"Dice\t = \t{dice}")

Dice	 = 	0.8794395185919782


In [ ]:
Y_test = Y_test.flatten()
Y_pred = Y_pred.flatten()

In [ ]:
np.unique(Y_test)

array([0.        , 0.00392157, 0.00784314, 0.01176471, 0.01568627,
       0.01960784, 0.02352941, 0.02745098, 0.03137255, 0.03529412,
       0.03921569, 0.04313725, 0.04705882, 0.05098039, 0.05490196,
       0.05882353, 0.0627451 , 0.06666667, 0.07058824, 0.0745098 ,
       0.07843137, 0.08235294, 0.08627451, 0.09019608, 0.09411765,
       0.09803922, 0.10196078, 0.10588235, 0.10980392, 0.11372549,
       0.11764706, 0.12156863, 0.1254902 , 0.12941176, 0.13333333,
       0.1372549 , 0.14117647, 0.14509804, 0.14901961, 0.15294118,
       0.15686275, 0.16078431, 0.16470588, 0.16862745, 0.17254902,
       0.17647059, 0.18039216, 0.18431373, 0.18823529, 0.19215686,
       0.19607843, 0.2       , 0.20392157, 0.20784314, 0.21176471,
       0.21568627, 0.21960784, 0.22352941, 0.22745098, 0.23137255,
       0.23529412, 0.23921569, 0.24313725, 0.24705882, 0.25098039,
       0.25490196, 0.25882353, 0.2627451 , 0.26666667, 0.27058824,
       0.2745098 , 0.27843137, 0.28235294, 0.28627451, 0.29019

In [ ]:
def image_metrics(Y_test, Y_pred, lim=0.5):
    Y_pred_bin = np.zeros_like(Y_test)
    idx = Y_pred > lim
    Y_pred_bin[idx] = 1

    auc = roc_auc_score(Y_test, Y_pred)
    f1 = f1_score(Y_test, Y_pred_bin)
    acc = accuracy_score(Y_test, Y_pred_bin)
    mat = confusion_matrix(Y_test, Y_pred_bin)

    TN = mat[0][0]
    FN = mat[1][0]
    TP = mat[1][1]
    FP = mat[0][1]

    sen = TP/(TP+FN)
    spe = TN/(TN+FP)

    return auc, f1, acc, sen, spe

In [ ]:
start = datetime.datetime.now()

In [ ]:
auc, f1, acc, sen, spe = image_metrics(Y_test, Y_pred, lim=0.5)

ValueError: ignored

In [ ]:
print(f"Accuracy = \t{acc}")
print(f"f1score\t = \t{f1}")
print(f"AUC\t = \t{auc}")
print(f"Dice\t = \t{dice}")
print(f"Sensitivity = \t{sen}")
print(f"Specificity = \t{spe}")

In [ ]:
end = datetime.datetime.now()

In [ ]:
diff = end - start

In [ ]:
start

In [ ]:
diff

In [ ]:
with open(f"{main_path}/Data/{dataset}/results_{str(datetime.datetime.now())[:-7]}.txt", "w") as ff:
    ff.write(f"Accuracy = \t{acc}\n")
    ff.write(f"f1score\t = \t{f1}\n")
    ff.write(f"AUC\t = \t{auc}\n")
    ff.write(f"Dice\t = \t{dice}\n")
    ff.write(f"Sensitivity = \t{sen}\n")
    ff.write(f"Specificity = \t{spe}\n")

In [ ]:
print("----------Fin----------")